In [4]:
import requests
from requests.auth import HTTPBasicAuth
import json
import pandas as pd
import keyring

In [11]:
jira_domain = 'https://forgeholidays.atlassian.net/'
email = 'deepankar.srigyan@sykescottages.co.uk'
project_key = 'DAB ORDER BY created DESC'

#Credential needs to be stored in window credential before running this.
api_token = keyring.get_password('JIRA_API', 'deepankar.srigyan@sykescottages.co.uk')


In [12]:
# path correction
if not jira_domain.startswith('https://'):
    jira_domain = 'https://' + jira_domain

# Create Jira API URL
url = f"{jira_domain}/rest/api/3/search"

# Query 
query = {
    'jql': f'project={project_key}',
    'maxResults': 10000  # Adjust based on your needs
}

try:
    # request
    response = requests.get(
        url,
        headers={'Content-Type': 'application/json'},
        params=query,
        auth=HTTPBasicAuth(email, api_token),
        verify=False  # SSL security off, need to work with IT so we do not need to do it False
    )

    # request URL (for debugging)
    print("Requested URL:", response.url)

    # errors
    response.raise_for_status()

    # Process the response
    issues = response.json()['issues']
    print(f'Total issues fetched: {len(issues)}')

    # Convert to DataFrame
    df_issues = pd.json_normalize(issues)  # JSON data into a  table

    # Save to an Excel file
    df_issues.to_excel('jira_issues.xlsx', index=False)  

    print('Issues have been saved to jira_issues.json and jira_issues.xlsx')  # Update print statement

except requests.exceptions.RequestException as e:
    print(f'An error occurred: {e}')

C:\Users\DeepankarSrigyan\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forgeholidays.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Requested URL: https://forgeholidays.atlassian.net//rest/api/3/search?jql=project%3DDAB+ORDER+BY+created+DESC&maxResults=1000
Total issues fetched: 100
Issues have been saved to jira_issues.json and jira_issues.xlsx
